API Examples
============

Some minimal, miscellaneous examples on how to use Sina. First, some basic setup: import modules, define datasets, and load some data.

In [ ]:
import IPython.display as ipyd
import ipywidgets as widgets
from sina.datastores.sql import DAOFactory
import sina.utils
import tabulate
import six

database = sina.utils.get_example_path('noaa/data.sqlite')
print("Using database {}".format(database))

type_to_query = "obs"

print("Creating data access object factory.")
factory = DAOFactory(db_path=database)
record_dao = factory.create_record_dao()

print("Loading the data from the database.  This may take a while.")
all_ids = list(record_dao.get_all_of_type(type_to_query, ids_only=True))

print("The data has loaded.  Now proceed to the next cell.")


"Get all files" dropdown
-----------------------------

Generate a dropdown selector for all available records. Display a table of all files associated with the selected record. Note that the default database only has one file per Record.

In [ ]:
column_display_names = {"mimetype": "Mimetype", "uri": "URI", "tags": "Tags"}

select_widget = widgets.Dropdown(
    options=all_ids,
    value=all_ids[0],
    description='Record:',
    disabled=False,
)


def display_table(list_pos):
    """Displays a table populated by all files associated with a record"""
    all_files = record_dao.get(all_ids[list_pos]).files
    # Files are stored in a dictionary, but tabulate expects a list.
    file_list = []
    for uri, file_data in six.iteritems(all_files):
        file_data["uri"] = uri
        file_list.append(file_data)
    tbl = tabulate.tabulate([column_display_names] + file_list, tablefmt='html')
    ipyd.display(ipyd.HTML(tbl))


def on_select(change):
    """Clears output, displays widget and file table"""
    ipyd.clear_output(wait=False)
    ipyd.display(select_widget)
    list_pos = change['owner'].index
    display_table(list_pos)


select_widget.observe(on_select)

ipyd.display(select_widget)
display_table(0)

Releasing Resources
-------------------------
Don't forget to release resources when you're all done!

In [ ]:
factory.close()